## Kombinasi 7 :
- Delete duplicate
- Impute missing value using iterative imputer
- Outlier handling with winsorization
- Encoding
- Standard Scaler
- Feature selection with Decision Tree

In [57]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [58]:
df = pd.read_csv("../UFC_train.csv")

df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [59]:
# Delete duplicate
print(f"Shape before dropping duplicate : {df.shape}")
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicate : {df.shape}")

Shape before dropping duplicate : (5410, 144)
Shape after dropping duplicate : (5410, 144)


In [60]:
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

In [61]:
check_null(df)

                         Total   Percent
R_Weight_lbs                 2  0.000370
R_Height_cms                 4  0.000739
B_Weight_lbs                 8  0.001479
B_Height_cms                10  0.001848
R_Stance                    27  0.004991
...                        ...       ...
B_avg_GROUND_landed       1293  0.239002
B_avg_GROUND_att          1293  0.239002
B_avg_opp_CLINCH_landed   1293  0.239002
B_avg_TD_att              1293  0.239002
B_avg_HEAD_landed         1293  0.239002

[109 rows x 2 columns]


In [62]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [63]:
# Impute missing value using iterative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(random_state=42)
df_imputed_number = imputer.fit_transform(df_number)
df_imputed_number = pd.DataFrame(df_imputed_number, columns=df_number.columns)

c:\Python312\Lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [64]:
# Drop R_fighter, B_fighter, Referee, location, date
df_object = df_object.drop(['R_fighter', 'B_fighter', 'Referee', 'location', 'date'], axis=1)

In [65]:
# Combine df_object and df_imputed_number
df = pd.concat([df_object, df_imputed_number], axis=1)
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,Red,0.086256,0.205955,0.389739,0.375177,1.213091,0.231261,...,0.0,0.0,0.0,0.0,0.0,182.88,177.80000,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,Blue,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,4.0,0.0,1.0,0.0,180.34,187.96000,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,Blue,0.350905,0.161033,0.497405,0.473810,-0.284956,0.286680,...,0.0,0.0,0.0,0.0,0.0,185.42,183.67835,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,Blue,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,...,1.0,5.0,6.0,3.0,0.0,177.80,185.42000,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,0.0,2.0,3.0,0.0,1.0,160.02,167.64000,135.0,34.0,31.0


In [66]:
check_null(df)

          Total   Percent
R_Stance     27  0.004991
B_Stance     64  0.011830


In [67]:
# Impute missing R_stance and B_stance with mode
df['R_Stance'].fillna(df['R_Stance'].mode()[0], inplace=True)
df['B_Stance'].fillna(df['B_Stance'].mode()[0], inplace=True)

In [68]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [69]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())

    return outliers

In [70]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [71]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                 397
B_avg_opp_KD             794
B_avg_SIG_STR_pct        386
B_avg_opp_SIG_STR_pct    351
B_avg_TD_pct             238
                        ... 
R_Height_cms              19
R_Reach_cms               76
R_Weight_lbs             301
B_age                     46
R_age                     77
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 7.33826247689464%
B_avg_opp_KD = 14.67652495378928%
B_avg_SIG_STR_pct = 7.1349353049907585%
B_avg_opp_SIG_STR_pct = 6.487985212569317%
B_avg_TD_pct = 4.399260628465805%
B_avg_opp_TD_pct = 6.210720887245841%
B_avg_SUB_ATT = 8.133086876155268%
B_avg_opp_SUB_ATT = 5.970425138632162%
B_avg_REV = 11.959334565619224%
B_avg_opp_REV = 11.44177449168207%
B_avg_SIG_STR_att = 5.360443622920517%
B_avg_SIG_STR_landed = 5.582255083179297%
B_avg_opp_SIG_STR_att = 5.046210720887246%
B_avg_opp_SIG_STR_landed = 4.7134935304990755%
B_avg_TOTAL_STR_att = 4.011090573012939%
B_avg_TOTAL_STR_

In [72]:
from scipy.stats.mstats import winsorize

# Identify columns with more than 0.5% outliers
outlier_columns = []
for key in outliers.keys():
    if outliers[key] / df_number[key].shape[0] > 0.005:
        outlier_columns.append(key)

# Winsorize the identified columns
for column in outlier_columns:
    df_number[column] = winsorize(df_number[column], limits=(0, 0.1))

# Combine df_object and df_number
df = pd.concat([df_object, df_number], axis=1)
df.head()


,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,Red,0.086256,0.205955,0.389739,0.375177,0.642656,0.231261,...,0.0,0.0,0.0,0.0,0.0,182.88,177.80000,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,Blue,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,4.0,0.0,1.0,0.0,180.34,187.96000,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,Blue,0.350905,0.161033,0.497405,0.473810,-0.284956,0.286680,...,0.0,0.0,0.0,0.0,0.0,185.42,183.67835,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,Blue,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,...,1.0,4.0,4.0,2.0,0.0,177.80,185.42000,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,0.0,2.0,3.0,0.0,0.0,160.02,167.64000,135.0,34.0,31.0


In [73]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

c:\Python312\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
c:\Python312\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


Outlier pada tiap atribut:
B_avg_KD                   0
B_avg_opp_KD             794
B_avg_SIG_STR_pct        177
B_avg_opp_SIG_STR_pct    136
B_avg_TD_pct              73
                        ... 
R_Height_cms              19
R_Reach_cms               35
R_Weight_lbs               0
B_age                      8
R_age                      6
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 0.0%
B_avg_opp_KD = 14.67652495378928%
B_avg_SIG_STR_pct = 3.2717190388170057%
B_avg_opp_SIG_STR_pct = 2.5138632162661736%
B_avg_TD_pct = 1.3493530499075785%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 0.0%
B_avg_opp_SUB_ATT = 0.0%
B_avg_REV = 11.959334565619224%
B_avg_opp_REV = 11.44177449168207%
B_avg_SIG_STR_att = 0.9981515711645103%
B_avg_SIG_STR_landed = 0.0%
B_avg_opp_SIG_STR_att = 0.0%
B_avg_opp_SIG_STR_landed = 0.0%
B_avg_TOTAL_STR_att = 0.0%
B_avg_TOTAL_STR_landed = 0.0%
B_avg_opp_TOTAL_STR_att = 0.0%
B_avg_opp_TOTAL_STR_landed = 0.0%
B_avg_TD_att = 0.0%
B_avg_TD_l

In [74]:
df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')

In [75]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,Middleweight,Orthodox,Southpaw,Red,0.086256,0.205955,0.389739,0.375177,0.642656,0.231261,...,0.0,0.0,0.0,0.0,182.88,177.80000,185.0,26.0,21.0,male
1,Lightweight,Orthodox,Orthodox,Blue,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,...,4.0,0.0,1.0,0.0,180.34,187.96000,170.0,27.0,30.0,male
2,Welterweight,Orthodox,Orthodox,Blue,0.350905,0.161033,0.497405,0.473810,-0.284956,0.286680,...,0.0,0.0,0.0,0.0,185.42,183.67835,170.0,28.0,31.0,male
3,Welterweight,Southpaw,Orthodox,Blue,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,...,4.0,4.0,2.0,0.0,177.80,185.42000,170.0,28.0,34.0,male
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,2.0,3.0,0.0,0.0,160.02,167.64000,135.0,34.0,31.0,male


In [76]:
df['weight_class'] = df['weight_class'].str.replace('Women', '')

In [77]:
# Get all weight_class values
weight_class = df['weight_class'].unique()
weight_class

array(['Middleweight', 'Lightweight', 'Welterweight', 'Bantamweight',
       'Flyweight', 'LightHeavyweight', 'Strawweight', 'Featherweight',
       'OpenWeight', 'Heavyweight', 'CatchWeight'], dtype=object)

In [78]:
# Create a dictionary to map weight_class values to numbers
weight_class_dict = {
    'CatchWeight' : 0,
    'Strawweight' : 1,
    'Flyweight' : 2,
    'Bantamweight' : 3,
    'Featherweight' : 4,
    'Lightweight' : 5,
    'Welterweight' : 6,
    'Middleweight' : 7,
    'LightHeavyweight' : 8,
    'Heavyweight' : 9,
    'OpenWeight' : 10,
}

gender_dict = {
    'male' : 1,
    'women' : 0
}

In [79]:
# Map each weight_class value to the correct number
df['weight_class'] = df['weight_class'].map(weight_class_dict)

In [80]:
# Map each gender value to the correct number
df['gender'] = df['gender'].map(gender_dict)

In [81]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,7,Orthodox,Southpaw,Red,0.086256,0.205955,0.389739,0.375177,0.642656,0.231261,...,0.0,0.0,0.0,0.0,182.88,177.80000,185.0,26.0,21.0,1
1,5,Orthodox,Orthodox,Blue,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,...,4.0,0.0,1.0,0.0,180.34,187.96000,170.0,27.0,30.0,1
2,6,Orthodox,Orthodox,Blue,0.350905,0.161033,0.497405,0.473810,-0.284956,0.286680,...,0.0,0.0,0.0,0.0,185.42,183.67835,170.0,28.0,31.0,1
3,6,Southpaw,Orthodox,Blue,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,...,4.0,4.0,2.0,0.0,177.80,185.42000,170.0,28.0,34.0,1
4,3,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,2.0,3.0,0.0,0.0,160.02,167.64000,135.0,34.0,31.0,1


In [82]:
# Get the remaining object columns
df_object = df.select_dtypes(include='object')

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_object)

# Combine the encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original object columns
df.drop(df_object.columns, axis=1, inplace=True)

df.head()


,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch,Winner_Blue,Winner_Draw,Winner_Red
0,7,0.086256,0.205955,0.389739,0.375177,0.642656,0.231261,0.706650,0.078682,0.147268,...,False,False,False,False,False,True,False,False,False,True
1,5,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,0.156250,0.132812,0.000000,...,False,False,False,True,False,False,False,True,False,False
2,6,0.350905,0.161033,0.497405,0.473810,-0.284956,0.286680,0.347275,0.637276,0.161916,...,False,False,False,True,False,False,False,True,False,False
3,6,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,0.093750,0.062500,0.000000,...,True,False,False,True,False,False,False,True,False,False
4,3,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.062500,0.000244,0.125000,...,False,False,False,True,False,False,False,False,False,True


In [83]:
# Export to CSV
df.to_csv('UFC_kombinasi7_wt_featureselection.csv', index=False)

In [84]:
from sklearn.tree import DecisionTreeRegressor

# Feature selection with Decision Tree
X = df.drop('B_Reach_cms', axis=1)
y = df['B_Reach_cms']

# Splitting data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

model = DecisionTreeRegressor()
model.fit(X_train, y_train)

c:\Python312\Lib\site-packages\pandas\core\dtypes\astype.py:134: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


ValueError: Input X contains infinity or a value too large for dtype('float32').

In [ ]:
feature_importances = model.feature_importances_

In [ ]:
top_n = 20
top_features_indices = feature_importances.argsort()[-top_n:][::-1]

In [ ]:
top_features_indices

array([ 64,  65,   9, 134, 132,  31, 130,   0,  17,   1,  86,  18,  62,
        22, 113,  47,  73,  39, 106,  41])

In [ ]:
# Get the names of the top features
top_features = X.columns[top_features_indices]
top_features

Index(['B_Height_cms', 'B_Weight_lbs', 'B_avg_REV', 'gender', 'B_age',
       'B_avg_LEG_att', 'R_Reach_cms', 'weight_class',
       'B_avg_opp_TOTAL_STR_att', 'B_avg_KD', 'R_avg_opp_TD_att',
       'B_avg_opp_TOTAL_STR_landed', 'B_win_by_Submission',
       'B_avg_opp_TD_landed', 'R_avg_opp_CTRL_time(seconds)',
       'B_avg_CTRL_time(seconds)', 'R_avg_opp_SUB_ATT', 'B_avg_CLINCH_att',
       'R_avg_opp_CLINCH_att', 'B_avg_opp_CLINCH_att'],
      dtype='object')

In [ ]:
# Create a dataframe with only the top features
df_top_features = df[top_features]
df_top_features['B_Reach_cms'] = df['B_Reach_cms']
df_top_features.head()

/var/folders/53/kv053qr13vxg2_y_1lg6hjg80000gn/T/ipykernel_97214/3681793381.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_features['B_Reach_cms'] = df['B_Reach_cms']


,B_Height_cms,B_Weight_lbs,B_avg_REV,gender,B_age,B_avg_LEG_att,R_Reach_cms,weight_class,B_avg_opp_TOTAL_STR_att,B_avg_KD,...,B_avg_opp_TOTAL_STR_landed,B_win_by_Submission,B_avg_opp_TD_landed,R_avg_opp_CTRL_time(seconds),B_avg_CTRL_time(seconds),R_avg_opp_SUB_ATT,B_avg_CLINCH_att,R_avg_opp_CLINCH_att,B_avg_opp_CLINCH_att,B_Reach_cms
0,182.88,185.0,0.200562,1,26.0,6.573702,177.800000,7,92.655464,0.186013,...,48.452845,0.0,1.056337,119.118031,129.196369,0.443638,8.063378,7.545087,7.493945,190.50
1,177.80,155.0,0.000000,1,27.0,6.523438,187.960000,5,91.882812,0.649799,...,33.078125,0.0,0.085938,115.000000,35.945312,0.312500,5.671875,4.562500,0.710938,177.80
2,182.88,170.0,0.182111,1,28.0,6.573702,189.230788,6,92.789902,0.211328,...,48.503628,0.0,1.121961,132.749212,141.719812,0.420586,8.063378,7.545087,7.493945,190.50
3,175.26,185.0,0.000000,1,28.0,3.421875,185.420000,6,52.046875,0.649799,...,28.671875,0.0,0.445312,116.292816,68.281250,0.007238,11.867188,6.983452,12.179688,175.26
4,165.10,135.0,0.125000,1,34.0,8.278564,167.640000,3,125.859863,0.500000,...,71.359619,2.0,1.380859,162.796875,209.931152,0.000000,9.979004,15.437500,14.500000,170.18


In [ ]:
df_top_features['B_Reach_cms'] = y

In [ ]:
# Export to CSV
df_top_features.to_csv('UFC_kombinasi7.csv', index=False)
